<ol>Mobility Features
<li>Stay points
<li>Entropy
<li>Normalized Entroy
<li>Percentage Home Stay
<li>Radius of Gyration
<li>Location Variance

seperate features for weekdays and weekends
<ol>
<li>Location variance
<li>Circadian movement
<li>Speed mean*
<li>Speed variance*
<li>Total distance
<li>Number of clusters/Stop points
<li>Entropy 
<br>
<br>
<b>Results.</b> The results of this study replicated our previous findings that a number of GPS features, including location variance, entropy, and circadian movement, were significantly correlated with PHQ-9 scores (r’s ranging from−0.43 to−0.46,p-values< .05). We also found that these relationships were stronger when GPS features were calculated from weekend, compared to weekday, data

In [1]:
from imports import *
import datetime
pd.set_option('display.max_rows', None)
np.seterr(divide = 'ignore')
# from lachesis import *

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
dataFilename1 = "Location_decrypted_temp.csv"

dataPath = "/csv/backup_frigg1"

In [3]:
# read file with a header
location = pd.read_csv(os.path.join(dataPath, dataFilename1), sep="|")
location.head()

,id,participant,attribute,latitude,longitude,address,timestamp,uploadtimestamp
0,61a97eceac44ca3f9df3a02c,PROSITC000189,Location,44.6422,-63.6264,"Rockwood Ave, Halifax, NS, B3N 1X5, Canada",2020-07-24 22:49:14,2020-07-25 00:04:47.478000
1,61a97eceac44ca3f9df3a02d,PROSITC000189,Location,44.6423,-63.6262,"Rockwood Ave, Halifax, NS, B3N 1X5, Canada",2020-07-24 22:49:26,2020-07-25 00:04:47.478000
2,61a97eceac44ca3f9df3a02e,PROSITC000189,Location,44.6421,-63.6265,"Rockwood Ave, Halifax, NS, B3N 1X5, Canada",2020-07-24 22:50:58,2020-07-25 00:04:47.478000
3,61a97eceac44ca3f9df3a02f,PROSITC000189,Location,44.6425,-63.6263,"Rockwood Ave, Halifax, NS, B3N 1X5, Canada",2020-07-24 22:51:59,2020-07-25 00:04:47.478000
4,61a97eceac44ca3f9df3a030,PROSITC000189,Location,44.6422,-63.6263,"Rockwood Ave, Halifax, NS, B3N 1X5, Canada",2020-07-24 22:52:16,2020-07-25 00:04:47.478000


In [4]:
#change time to Halifax time
location["timestamp"] = pd.to_datetime(location["timestamp"], utc=True)
location["timestamp"] = pd.to_datetime(location["timestamp"]).dt.tz_convert(tz='America/Halifax')
location["timestamp"] = pd.to_datetime(location["timestamp"], utc=False)

#add new columns to help extract features
# location["date"] = pd.to_datetime(location["timestamp"]).dt.date
# location["weekday"] = pd.to_datetime(location["timestamp"]).dt.weekday    # Monday=0, Sunday=6
# location["time"] = pd.to_datetime(location["timestamp"]).dt.strftime('%H:%M:%S')

# sort data, remove duplicates and drop unecessary columns
location = location.sort_values(["participant", "timestamp"]).reset_index(drop=True)
location.drop_duplicates(subset=["participant", "timestamp", "latitude", "longitude", "address"], keep="last", inplace=True)
location.drop(["id", "attribute", "address", "uploadtimestamp"], axis=1, inplace=True)

location.head()

,participant,latitude,longitude,timestamp
0,PROSIT0003,44.772200,-63.694800,2020-07-24 12:44:52-03:00
1,PROSIT0003,44.772502,-63.675176,2020-07-24 12:44:55-03:00
2,PROSIT0003,44.772149,-63.675171,2020-07-24 12:44:57-03:00
3,PROSIT0003,44.771925,-63.674963,2020-07-24 12:44:59-03:00
4,PROSIT0003,44.771762,-63.674843,2020-07-24 12:45:00-03:00


In [5]:
def get_stayPoints(df, roaming_distance=50, minimum_stay=10, number_jobs=1):
    """
    method returns the total number of stay points for the given location points
    uses the logic used in Microsoft paper
    """
    # Parameters
    roaming_distance = meters2degrees(roaming_distance) # 50 meters converted to degrees
#     minimum_stay  # minutes
#     number_jobs   # number of parallel jobs

    # Set index 
    df = df.reset_index().set_index(["participant", "timestamp"])

    # Call helper-function to process entire df in one go
    df_stops = process_data(df=df, 
                            roam_dist=roaming_distance, 
                            min_stay=minimum_stay, 
                            n_jobs=number_jobs,
                            print_output="notebook")
    df_stops = pd.concat(df_stops)

    # Only keep users with more than one stop
    df_stops = (df_stops
        .groupby("participant").filter(lambda x: len(x) > 1)
        .set_index(["participant"]))

    return df_stops


def get_clusters(df_stops, distance=100):
    """
    method returns the total number of clusters for the given stay points for the given location points
    uses the logic used in Microsoft paper
    need to see if i can merge this with above method
    """
    # Clustering parameters
    linkage_method = 'centroid'
    distance = meters2degrees(distance)

    # Cluster stoplocations on a per user basis
    df_clusters = (df_stops.groupby('participant')
                  .apply(lambda x: 
                         cluster_stoplocations(x, 'centroid', distance))
                  .reset_index())

    return df_clusters

In [6]:
df_stops = get_stayPoints(location)

df_clusters = get_clusters(df_stops)
df_clusters = df_clusters.sort_values(["participant", "timestamp"]).reset_index(drop=True)

df_clusters["duration"] = (df_clusters["t_end"] - df_clusters["t_start"]).astype('timedelta64[m]')
df_clusters["date"] = pd.to_datetime(df_clusters["timestamp"]).dt.date
df_clusters["weekday"] = pd.to_datetime(df_clusters["timestamp"]).dt.weekday    # Monday=0, Sunday=6

df_clusters.head(10)

,participant,timestamp,latitude,longitude,t_start,t_end,cluster_assignment,duration,date,weekday
0,PROSIT0003,2020-07-24 17:17:24-03:00,44.782624,-63.688884,2020-07-24 17:17:24-03:00,2020-07-24 17:33:11-03:00,1,15.0,2020-07-24,4
1,PROSIT0003,2020-07-25 10:17:19-03:00,44.673163,-63.525412,2020-07-25 10:17:19-03:00,2020-07-25 13:32:35-03:00,2,195.0,2020-07-25,5
2,PROSIT0003,2020-07-25 13:32:45-03:00,44.708689,-63.581347,2020-07-25 13:32:45-03:00,2020-07-25 13:53:04-03:00,3,20.0,2020-07-25,5
3,PROSIT0003,2020-07-25 14:22:09-03:00,44.782644,-63.688881,2020-07-25 14:22:09-03:00,2020-07-25 14:33:25-03:00,1,11.0,2020-07-25,5
4,PROSIT0025,2020-07-24 21:42:15-03:00,44.953691,-63.570078,2020-07-24 21:42:15-03:00,2020-07-24 21:55:55-03:00,2,13.0,2020-07-24,4
5,PROSIT0025,2020-07-25 14:38:03-03:00,44.683351,-63.548926,2020-07-25 14:38:02-03:00,2020-07-25 15:19:14-03:00,1,41.0,2020-07-25,5
6,PROSIT00A,2020-07-24 23:46:29-03:00,49.820500,9.971200,2020-07-24 21:01:39-03:00,2020-07-24 23:46:29-03:00,1,164.0,2020-07-24,4
7,PROSIT00A,2020-07-25 00:23:05-03:00,49.820400,9.970700,2020-07-25 00:23:05-03:00,2020-07-25 01:36:59-03:00,1,73.0,2020-07-25,5
8,PROSIT00A,2020-07-25 03:12:35-03:00,49.820000,9.970200,2020-07-25 03:12:35-03:00,2020-07-25 03:46:34-03:00,1,33.0,2020-07-25,5
9,PROSIT0231,2020-07-25 15:54:21-03:00,40.899200,-73.870700,2020-07-25 02:13:57-03:00,2020-07-25 15:54:21-03:00,1,820.0,2020-07-25,5


In [9]:
location.head(50)

,participant,latitude,longitude,timestamp
0,PROSIT0003,44.772200,-63.694800,2020-07-24 12:44:52-03:00
1,PROSIT0003,44.772502,-63.675176,2020-07-24 12:44:55-03:00
2,PROSIT0003,44.772149,-63.675171,2020-07-24 12:44:57-03:00
3,PROSIT0003,44.771925,-63.674963,2020-07-24 12:44:59-03:00
4,PROSIT0003,44.771762,-63.674843,2020-07-24 12:45:00-03:00
5,PROSIT0003,44.771521,-63.674638,2020-07-24 12:45:02-03:00
6,PROSIT0003,44.771500,-63.674600,2020-07-24 12:53:54-03:00
7,PROSIT0003,44.768598,-63.669017,2020-07-24 12:53:55-03:00
8,PROSIT0003,44.769089,-63.669775,2020-07-24 12:54:00-03:00
9,PROSIT0003,44.769100,-63.669800,2020-07-24 12:54:01-03:00


In [8]:
# columns=["participant", "date", "weekday", "noOfStayPoints", "locationVariance", "entropy", "normalizedEntropy", "homeStay", "radiusOfGyration"]
mobilityFeatures = pd.DataFrame()

In [9]:
# get no of Stay points in a day
mobilityFeatures[["participant", "date", "weekday", "noOfStayPoints"]] = df_clusters.groupby(["participant", "date", "weekday"]).size().reset_index(name="noOfStayPoints")
mobilityFeatures.head()

,participant,date,weekday,noOfStayPoints
0,PROSIT0003,2020-07-24,4,1
1,PROSIT0003,2020-07-25,5,3
2,PROSIT0025,2020-07-24,4,1
3,PROSIT0025,2020-07-25,5,1
4,PROSIT00A,2020-07-24,4,1


In [10]:
# get Location Variance
df_locationVariance = df_clusters.groupby(["participant", "date"]).apply(lambda x: np.log10(x["latitude"].var()**2 + x["longitude"].var()**2)).reset_index(name="locationVariance")
mobilityFeatures = pd.merge(mobilityFeatures, df_locationVariance, on=["participant", "date"], how="left")
mobilityFeatures.head()

,participant,date,weekday,noOfStayPoints,locationVariance
0,PROSIT0003,2020-07-24,4,1,NaN
1,PROSIT0003,2020-07-25,5,3,-4.241278
2,PROSIT0025,2020-07-24,4,1,NaN
3,PROSIT0025,2020-07-25,5,1,NaN
4,PROSIT00A,2020-07-24,4,1,NaN


In [11]:
# get cluster where max time is spent i.e. Home
df_home = df_clusters.groupby(["participant", "cluster_assignment"])["duration"].sum().reset_index()
df_home_temp = df_home.groupby(["participant"])["duration"].max().reset_index()
df_home = pd.merge(df_home, df_home_temp[["participant", "duration"]], on=["participant", "duration"], how="inner")
df_home["home"] = 1
df_home.drop("duration", axis=1, inplace=True)

#  get homestay
df_clusters = pd.merge(df_clusters, df_home, on=["participant", "cluster_assignment"], how="left")
df_clusters.fillna(0, inplace=True)

df_home_stay = df_clusters.loc[df_clusters["home"]==1, :].groupby(["participant", "date"])["duration"].sum().reset_index(name="homeStay")
df_home_stay["homeStay"] = round(df_home_stay["homeStay"] * 100 / (24*60), 2)

mobilityFeatures = pd.merge(mobilityFeatures, df_home_stay, on=["participant", "date"], how="left")
mobilityFeatures.head()

,participant,date,weekday,noOfStayPoints,locationVariance,homeStay
0,PROSIT0003,2020-07-24,4,1,NaN,NaN
1,PROSIT0003,2020-07-25,5,3,-4.241278,13.54
2,PROSIT0025,2020-07-24,4,1,NaN,NaN
3,PROSIT0025,2020-07-25,5,1,NaN,2.85
4,PROSIT00A,2020-07-24,4,1,NaN,11.39


In [12]:
# Entropy
df_entropy = df_clusters.groupby(["participant", "date"])["duration"].apply(lambda x : sum(x*np.log10(x))).reset_index(name="entropy")
mobilityFeatures = pd.merge(mobilityFeatures, df_entropy, on=["participant", "date"], how="left")
mobilityFeatures.head()

,participant,date,weekday,noOfStayPoints,locationVariance,homeStay,entropy
0,PROSIT0003,2020-07-24,4,1,NaN,NaN,17.641369
1,PROSIT0003,2020-07-25,5,3,-4.241278,13.54,484.032669
2,PROSIT0025,2020-07-24,4,1,NaN,NaN,14.481264
3,PROSIT0025,2020-07-25,5,1,NaN,2.85,66.124138
4,PROSIT00A,2020-07-24,4,1,NaN,11.39,363.234391


In [13]:
# Normalized Entropy
df_N = df_clusters.groupby(["participant", "date"])["cluster_assignment"].size().reset_index(name="N")
mobilityFeatures = pd.merge(mobilityFeatures, df_N, on=["participant", "date"], how="left")
mobilityFeatures.head()

,participant,date,weekday,noOfStayPoints,locationVariance,homeStay,entropy,N
0,PROSIT0003,2020-07-24,4,1,NaN,NaN,17.641369,1
1,PROSIT0003,2020-07-25,5,3,-4.241278,13.54,484.032669,3
2,PROSIT0025,2020-07-24,4,1,NaN,NaN,14.481264,1
3,PROSIT0025,2020-07-25,5,1,NaN,2.85,66.124138,1
4,PROSIT00A,2020-07-24,4,1,NaN,11.39,363.234391,1


In [14]:
mobilityFeatures["normalizedEntropy"] = mobilityFeatures["entropy"] / mobilityFeatures["N"]
mobilityFeatures.head()

,participant,date,weekday,noOfStayPoints,locationVariance,homeStay,entropy,N,normalizedEntropy
0,PROSIT0003,2020-07-24,4,1,NaN,NaN,17.641369,1,17.641369
1,PROSIT0003,2020-07-25,5,3,-4.241278,13.54,484.032669,3,161.344223
2,PROSIT0025,2020-07-24,4,1,NaN,NaN,14.481264,1,14.481264
3,PROSIT0025,2020-07-25,5,1,NaN,2.85,66.124138,1,66.124138
4,PROSIT00A,2020-07-24,4,1,NaN,11.39,363.234391,1,363.234391


In [16]:
# Get medoid of each destination (cluster)
df_clustermedoids = (df_clusters.groupby('participant')
    .apply(lambda x: get_clustermedoids(x))
    .reset_index(drop=True))

# Compute stop counts at each destination
df_clustersizes = (df_clusters
                   .groupby(['participant', 'date', 'cluster_assignment'])
                   .apply(lambda x: len(x))
                   .reset_index(name='count'))

# Merge medoids and counts
df_destinations = pd.merge(df_clustermedoids.loc[:,['participant', 'date', 'timestamp',
                                                 'latitude', 'longitude',
                                                 'cluster_assignment']], 
                             df_clustersizes, 
                             on=['participant', 'date', 'cluster_assignment'], 
                             how='left')

# Export and preview data
# df_destinations

In [17]:
# Cluster stoplocations on a per user basis
df_rgyration = (df_destinations
                .loc[:,['participant', 'date', 'longitude', 'latitude', 'count']]
                .groupby(['participant', 'date'])
                .apply(lambda x: rgiration_at_k(x, k=None))
                .reset_index())

df_rgyration.columns = ['participant', 'date', 'radius_gyration']
df_rgyration.head()

,participant,date,radius_gyration
0,PROSIT0003,2020-07-24,0.000000
1,PROSIT0003,2020-07-25,3688.216552
2,PROSIT0025,2020-07-24,0.000000
3,PROSIT0025,2020-07-25,0.000000
4,PROSIT00A,2020-07-25,0.000000


In [18]:
mobilityFeatures = pd.merge(mobilityFeatures, df_rgyration, on=["participant", "date"], how="left")
mobilityFeatures.head()

,participant,date,weekday,noOfStayPoints,locationVariance,homeStay,entropy,N,normalizedEntropy,radius_gyration
0,PROSIT0003,2020-07-24,4,1,NaN,NaN,17.641369,1,17.641369,0.000000
1,PROSIT0003,2020-07-25,5,3,-4.241278,13.54,484.032669,3,161.344223,3688.216552
2,PROSIT0025,2020-07-24,4,1,NaN,NaN,14.481264,1,14.481264,0.000000
3,PROSIT0025,2020-07-25,5,1,NaN,2.85,66.124138,1,66.124138,0.000000
4,PROSIT00A,2020-07-24,4,1,NaN,11.39,363.234391,1,363.234391,NaN


In [20]:
print("end")

end
